In [88]:
def parse_lines(lines):
    node_map = {}
    for line in lines:
        source, targets = line.split(" -> ")
        tmp_map = {}
        state = "none"
        # get source & source type
        source_type = "none"
        if source == "broadcaster":
            source_type = "broadcaster"
        elif source.startswith("%"):
            source_type = "flipflop"
            source = source[1:]
            state = "off"
            tmp_map["state"] = state
        elif source.startswith("&"):
            source_type = "conjunction"
            source = source[1:]
            memory = {}
            tmp_map["memory"] = memory
        # build targets list
        targets = targets.split(", ")
        # add to node map
        node_map[source] = {}
        node_map[source]["type"] = source_type
        node_map[source]["targets"] = targets
        node_map[source]["from"] = [] 
        node_map[source]["name"] = source
        for k,v in tmp_map.items():
            node_map[source][k] = v

    # check if all targets exist
    new_node_map = {}
    for node, node_info in node_map.items():
        for target in node_info["targets"]:
            if target not in node_map:
                new_node_map[target] = {}
                new_node_map[target]["type"] = "none"
                new_node_map[target]["targets"] = []
                new_node_map[target]["from"] = []
                new_node_map[target]["name"] = target
                print(f"Node {target} not found, creating...")
    node_map.update(new_node_map)

    # also build a from list
    for node, node_info in node_map.items():
        for target in node_info["targets"]:
            if target not in node_map:
                raise Exception("Target not found")
            node_map[target]["from"].append(node)

    # update initial memory for conjunctions
    for node, node_info in node_map.items():
        if node_info["type"] == "conjunction":
            for from_node in node_info["from"]:
                node_info["memory"][from_node] = "off"
    return node_map
        

In [90]:
def process_broadcast(node, from_node, pulse):
    events = []
    for target in node["targets"]:
        # event: source, target, pulse
        # just continue redirect
        events.append((node["name"], target, pulse))
    return events

In [91]:
def process_flipflop(node, from_node, pulse):
    events = []

    if pulse == "high":
        # nothing happens
        return events
    elif pulse == "low":
        # flip state
        if node["state"] == "off":
            node["state"] = "on"
        elif node["state"] == "on":
            node["state"] = "off"
        else:
            raise Exception(f"Invalid state {node['state']}")


        # send out
        out_pulse = "high" if node["state"] == "on" else "low"
        for target in node["targets"]:
            # event: source, target, pulse
            events.append((node["name"], target, out_pulse))
        return events

In [92]:
def process_none(node, from_node, pulse):
    if pulse == "high":
        return []
    elif pulse == "low":
        return [("!", "!", "low")]

In [93]:
def process_conjunction(node, from_node, pulse):
    # first update memory for that input
    node["memory"][from_node] = pulse

    events = []
    # if high on all memory, send out low
    if all([v == "high" for v in node["memory"].values()]):
        for target in node["targets"]:
            # event: source, target, pulse
            events.append((node["name"], target, "low"))
        return events
    else:
        # send a high to all targets
        for target in node["targets"]:
            # event: source, target, pulse
            events.append((node["name"], target, "high"))
        return events

In [106]:
with open("day20.txt", "r") as f:
    lines = [line.strip() for line in f.readlines()]
node_map = parse_lines(lines)

Node rx not found, creating...


In [107]:
total_high_pulse_count = 0
total_low_pulse_count = 0

last_level_hits = {
    "rz":[],
    "lf": [],
    "br": [],
    "fk": [],
}

for run_idx in range(100000):
    if run_idx % 10000 == 0:
        print(f"Run {run_idx}")
    queue = []
    queue.append(("button", "broadcaster", "low"))

    iter_step = 0
    while len(queue) > 0 and iter_step < 100000:
        if iter_step > 80000:
            print(f"Run {run_idx}, iter {iter_step}")
        iter_step += 1
        source, target, pulse = queue.pop(0)

        if source == "!" and target == "!":
            print("ok at run idx", run_idx)
            break
        # if source == "rz" and target == "lb" and pulse == "high":
        #     debug_hit.append(run_idx)
        #     print("node sending...", "run_idx", run_idx)
        #     print(f"{source} --({pulse})--> {target}")
        if source in last_level_hits and target == "lb" and pulse == "high":
            last_level_hits[source].append(run_idx)
            # print("node sending...", "run_idx", run_idx)
            # print(f"{source} --({pulse})--> {target}")

        if pulse == "high":
            total_high_pulse_count += 1
        elif pulse == "low":
            total_low_pulse_count += 1

        node = node_map[target]
        # print("node: ", node)
        if node["type"] == "broadcaster":
            queue.extend(process_broadcast(node, source, pulse))
        elif node["type"] == "flipflop":
            queue.extend(process_flipflop(node, source, pulse))
        elif node["type"] == "none":
            queue.extend(process_none(node, source, pulse))
        elif node["type"] == "conjunction":
            queue.extend(process_conjunction(node, source, pulse))
        else:
            raise Exception(f"Invalid node type {node['type']}")
            # print(f"Queue: {queue}")

# total_high_pulse_count * total_low_pulse_count

Run 0
Run 10000
Run 20000
Run 30000
Run 40000
Run 50000
Run 60000
Run 70000
Run 80000
Run 90000


In [109]:
last_level_hits

{'rz': [4056,
  8113,
  12170,
  16227,
  20284,
  24341,
  28398,
  32455,
  36512,
  40569,
  44626,
  48683,
  52740,
  56797,
  60854,
  64911,
  68968,
  73025,
  77082,
  81139,
  85196,
  89253,
  93310,
  97367],
 'lf': [3910,
  7821,
  11732,
  15643,
  19554,
  23465,
  27376,
  31287,
  35198,
  39109,
  43020,
  46931,
  50842,
  54753,
  58664,
  62575,
  66486,
  70397,
  74308,
  78219,
  82130,
  86041,
  89952,
  93863,
  97774],
 'br': [3876,
  7753,
  11630,
  15507,
  19384,
  23261,
  27138,
  31015,
  34892,
  38769,
  42646,
  46523,
  50400,
  54277,
  58154,
  62031,
  65908,
  69785,
  73662,
  77539,
  81416,
  85293,
  89170,
  93047,
  96924],
 'fk': [4078,
  8157,
  12236,
  16315,
  20394,
  24473,
  28552,
  32631,
  36710,
  40789,
  44868,
  48947,
  53026,
  57105,
  61184,
  65263,
  69342,
  73421,
  77500,
  81579,
  85658,
  89737,
  93816,
  97895]}

In [111]:
# one order diff of debug_hit
# diff_1 = [debug_hit[i+1] - debug_hit[i] for i in range(len(debug_hit)-1)]
# diff_1

for k,v in last_level_hits.items():
    diff_1 = [v[i+1] - v[i] for i in range(len(v)-1)]
    print(k, v[0], diff_1[0] ,diff_1)

rz 4056 4057 [4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057, 4057]
lf 3910 3911 [3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911, 3911]
br 3876 3877 [3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877, 3877]
fk 4078 4079 [4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079, 4079]


In [116]:
# find the smallest common multiple of all diff_1
from functools import reduce
def scm(numbers):
    # find the smallest common multiple of all numbers
    # https://stackoverflow.com/questions/147515/least-common-multiple-for-3-or-more-numbers
    def gcd(a, b):
        while b:
            a, b = b, a % b
        return a

    def lcm(a, b):
        return a * b // gcd(a, b)

    def lcmm(*args):
        return reduce(lcm, args)

    return lcmm(*numbers)

In [118]:
scm([4057,3911,3877,4079])

250924073918341

In [55]:
node_map['rx']

{'type': 'none', 'targets': [], 'from': ['lb'], 'name': 'rx'}

In [56]:
node_map['lb']

{'type': 'conjunction',
 'targets': ['rx'],
 'from': ['rz', 'lf', 'br', 'fk'],
 'name': 'lb',
 'memory': {'rz': 'low', 'lf': 'low', 'br': 'low', 'fk': 'low'}}

In [57]:
node_map['rz']

{'type': 'conjunction',
 'targets': ['lb'],
 'from': ['th'],
 'name': 'rz',
 'memory': {'th': 'high'}}

In [58]:
node_map['th']

{'type': 'conjunction',
 'targets': ['mj', 'rz', 'np', 'fq', 'cm'],
 'from': ['xb', 'gp', 'mj', 'fm', 'xd', 'hh', 'sx', 'dq', 'dj'],
 'name': 'th',
 'memory': {'xb': 'off',
  'gp': 'off',
  'mj': 'low',
  'fm': 'off',
  'xd': 'off',
  'hh': 'off',
  'sx': 'off',
  'dq': 'off',
  'dj': 'off'}}

In [59]:
len(node_map)

59